In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import numpy
import requests
from WalkScore import *
import copy
from create_date_features import create_date_features
from create_weather_features import create_weather_features


Distribution of Months
5    246662
4    245486
6        46
Name: tpep_dropoff_datetime_months, dtype: int64
Distribution of Holidays
0    486831
1      5363
Name: tpep_dropoff_datetime_is_holiday, dtype: int64
2016 1 1
2016 1 2
2016 1 3
2016 1 4
2016 1 5
2016 1 6
2016 1 7
2016 1 8
2016 1 9
2016 1 10
2016 1 11
2016 1 12
2016 1 13
2016 1 14
2016 1 15
2016 1 16
2016 1 17
2016 1 18
2016 1 19
2016 1 20
2016 1 21
2016 1 22
2016 1 23
2016 1 24
2016 1 25
2016 1 26
2016 1 27
2016 1 28
2016 1 29
2016 1 30
2016 1 31
2016 2 1
2016 2 2
2016 2 3
2016 2 4
2016 2 5
2016 2 6
2016 2 7
2016 2 8
2016 2 9
2016 2 10
2016 2 11
2016 2 12
2016 2 13
2016 2 14
2016 2 15
2016 2 16
2016 2 17
2016 2 18
2016 2 19
2016 2 20
2016 2 21
2016 2 22
2016 2 23
2016 2 24
2016 2 25
2016 2 26
2016 2 27
2016 2 28
2016 2 29
2016 3 1
2016 3 2
2016 3 3
2016 3 4
2016 3 5
2016 3 6
2016 3 7
2016 3 8
2016 3 9
2016 3 10
2016 3 11
2016 3 12
2016 3 13
2016 3 14
2016 3 15
2016 3 16
2016 3 17
2016 3 18
2016 3 19
2016 3 20
2016 3 21
2016 3 

In [42]:
from geopy.geocoders import GoogleV3

In [12]:
cab_data_p1 = pd.read_csv("cab_data_p1.csv")
cab_data_p2 = pd.read_csv("cab_data_p2.csv")
cab_data = pd.read_csv("cab_data.csv")

In [13]:
#Removes columns we dont need, feel free to change this based on what features we want/need
def scrub_df(df):
    del df['extra']
    del df['fare_amount']
    del df['improvement_surcharge']
    del df['mta_tax']
    del df['passenger_count']
    del df['payment_type']
    del df['ratecodeid']
    del df['store_and_fwd_flag']
    del df['tip_amount']
    del df['tolls_amount']
    del df['trip_distance']
    del df['vendorid']
    
    return df

In [14]:
#Clean the input dataset of columns we don't care about
basic_df = scrub_df(cab_data)
basic_df.to_csv('clean_cab_data.csv')
basic_df_p1 = scrub_df(cab_data_p1)
basic_df_p2 = scrub_df(cab_data_p2)

In [15]:
#Pairs the lat/lon columns into one column (for walkscore/geolocator)
def get_latlng(df):
    def f(x,y):
        coord = "%f, %f" % (x, y)
        return coord
    
    df['start_latlng'] = df[['dropoff_latitude', 'dropoff_longitude']].apply(lambda x: f(*x), axis=1)
    df['end_latlng'] = df[['pickup_latitude', 'pickup_longitude']].apply(lambda x: f(*x), axis=1)
    
    return df

df_with_latlng = get_latlng(basic_df)

In [16]:
#REMOVE WHEN UPLOADING
apiKey = 
walkscore = WalkScore(apiKey)
geolocator = Nominatim()

In [17]:
#Given input dataframe, computes distance of trip using geolocator and start/end lat/lon positions
def get_dist(df):

    def get_d(x,y):
        return geodesic(x, y).miles

    df['distance'] = df[['start_latlng', 'end_latlng']].apply(lambda x: get_d(*x), axis=1)
    
    return df

df_with_dist = get_dist(df_with_latlng)

In [19]:
df_with_date = create_date_features(df_with_dist)
print(df_with_date)

        Unnamed: 0  dropoff_latitude  dropoff_longitude  pickup_latitude  \
0                0         40.777241         -73.951744        40.776424   
1                1         40.777611         -73.951080        40.789371   
2                2         40.770386         -73.966476        40.761303   
3                3         40.767288         -73.966675        40.764381   
4                4         40.744068         -73.988991        40.754742   
5                5         40.787369         -73.954048        40.771824   
6                6         40.777973         -73.955894        40.759167   
7                7         40.737041         -73.988632        40.746235   
8                8         40.756405         -73.968155        40.761028   
9                9         40.805321         -73.961723        40.819996   
10              10         40.714500         -73.992882        40.725052   
11              11         40.755898         -73.975319        40.748158   
12          

In [20]:
df_with_weather = create_weather_features(df_with_date)
print(df_with_weather)

        Unnamed: 0  dropoff_latitude  dropoff_longitude  pickup_latitude  \
0                0         40.777241         -73.951744        40.776424   
1                1         40.777611         -73.951080        40.789371   
2                2         40.770386         -73.966476        40.761303   
3                3         40.767288         -73.966675        40.764381   
4                4         40.744068         -73.988991        40.754742   
5                5         40.787369         -73.954048        40.771824   
6                6         40.777973         -73.955894        40.759167   
7                7         40.737041         -73.988632        40.746235   
8                8         40.756405         -73.968155        40.761028   
9                9         40.805321         -73.961723        40.819996   
10              10         40.714500         -73.992882        40.725052   
11              11         40.755898         -73.975319        40.748158   
12          

In [21]:
#For now, write to CSV file, once we figure out how to make enough geo queries, add walkscore feature
df_with_weather.to_csv('data_with_features.csv')

In [34]:
#SLOW
def get_address(df):
    
    def reverse_geocode(latlng):
        res = geolocator.reverse(latlng)
        return res.address
    
    #VERY SLOW
    df['pickup_address'] = df['start_latlng'].apply(reverse_geocode)
    df['dropoff_address'] = df['end_latlng'].apply(reverse_geocode)    
    
    return df

df_with_address = get_address(res_df)

In [8]:
#Given part 1 and part 2 of data, returns a DF of walkability score for 
#Start and end coord
#SLOW
def get_walkability_score(df):
    def ws_score(lat,lon, address):
        score = walkscore.makeRequest(address, lat, lon)
        return score['walkscore']
    
    df['pickup_addr'] = df[['pickup_latitude', 'pickup_longitude', 'pickup_address']].apply(lambda x: ws_score(*x), axis=1)
    df['dropoff_addr'] = df[['dropoff_latitude', 'dropoff_longitude', 'dropoff_address']].apply(lambda x: ws_score(*x), axis=1)
    
    return df

In [44]:
print("Getting addresses")
#df_with_walkscore = get_walkability_score(df_with_address)

Getting addresses


In [34]:
print("Getting walkability scores")
start_w, end_w = get_walkability_score(res_df)
print(start_w)
print(end_w)

Getting walkability scores
0     100
1     100
2     100
3      99
4     100
5      99
6      99
7      98
8      99
9     100
10     98
11     98
12     98
13     57
14    100
15     99
16     99
17     99
18     99
19     98
dtype: int64
0      99
1      93
2     100
3      99
4     100
5      99
6      99
7      99
8      99
9     100
10     99
11     97
12     99
13     99
14     98
15    100
16    100
17     99
18     65
19     98
dtype: int64
